# Import dataset from kaggle

In [ ]:
!kaggle datasets download yutkin/corpus-of-russian-news-articles-from-lenta

 98% 569M/584M [00:08<00:00, 81.8MB/s]
100% 584M/584M [00:08<00:00, 72.5MB/s]


In [ ]:
!unzip "/content/corpus-of-russian-news-articles-from-lenta.zip"

Archive:  /content/corpus-of-russian-news-articles-from-lenta.zip
  inflating: lenta-ru-news.csv       


# Install Requirements

In [ ]:
!pip install transformers 

     |████████████████████████████████| 757kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 9.0MB/s 
     |████████████████████████████████| 890kB 33.2MB/s 
     |████████████████████████████████| 1.1MB 41.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9236785b3ab2a7e0f34872bf8f86d6e06099af0abe1ac75243733ef2e5eb89ff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Main



In [ ]:
import pandas as pd
data = pd.read_csv('/content/lenta-ru-news.csv', usecols=['title','text'], skiprows=range(1,6))
data.head()

,title,text
0,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...
1,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...
2,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...
3,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...
4,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...


In [ ]:
len(data)

800970

In [ ]:
with open('news-text.txt','w') as f:
    for x in data['text']:
        f.write(str(x))

In [ ]:
import numpy as np
pool = 100
bk = int(np.ceil(len(data)/pool))
for i in range(pool):
    filename = "/content/drive/My Drive/RuBART/texts/news-texts[" + str(i) + "].txt"
    with open(filename,'w') as f:
        for text in data['text'][i:i+bk]:
            f.write(str(text) + '\n')

Training tokenizer

In [ ]:
import torch

from transformers import BartTokenizer, BartForConditionalGeneration
from IPython.display import display, Markdown

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)

cuda


In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files='/content/news-text.txt', show_progress=True, vocab_size=80000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"
])

In [ ]:
tokenizer.save_model('/content/drive/My Drive/RuBART/RuBART-Lenta')

['/content/drive/My Drive/RuBART/RuBART-Lenta/vocab.json',
 '/content/drive/My Drive/RuBART/RuBART-Lenta/merges.txt']

Shorten text news - 256 token max

In [ ]:
mask = []
for text in data['text']:
    mask.append(len(tokenizer(str(text)).input_ids) <= 256)

In [ ]:
len(mask)

800970

In [ ]:
pd.Series(mask).value_counts()

True     491675
False    309295
dtype: int64

In [ ]:
short_data = data.loc[mask]

In [ ]:
len(short_data)

491675

In [ ]:
short_data.to_csv("/content/drive/My Drive/RuBART/news-short.csv", index=False)

Training BART model

In [4]:
from transformers import BartTokenizerFast
tokenizer = BartTokenizerFast.from_pretrained("/content/drive/MyDrive/Infosystems/RuBART/RuBART-Lenta")

In [5]:
a = tokenizer("Съешь ещё этих мягких французских булочек")
for i in a.input_ids:
    print(tokenizer.decode([i]))

<s>
С
ъ
ешь
 ещё
 этих
 мягких
 французских
 бул
очек
</s>


In [16]:
from transformers import AutoConfig, AutoModel
config = AutoConfig.from_pretrained('facebook/bart-large', vocab_size=80000)
model =  AutoModel.from_config(config)

In [17]:
config

BartConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel",
    "BartForConditionalGeneration",
    "BartForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": false,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddi

In [14]:
model.num_parameters()

406291456

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="",
    block_size=128,
)


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./RuBART",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

In [ ]:
trainer.save_model("./RuBART")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./RuBART",
    tokenizer="./RuBART"
)

In [ ]:
fill_mask("")

In [ ]:
from transformers import BartForConditionalGeneration

config = BartConfig.from_json_file()
model = BartForConditionalGeneration(config=config)

In [ ]:
inputs = tokenizer.batch_encode_plus([article_text], max_length=1024, return_tensors='pt')

pp = pprint.PrettyPrinter(indent=0, width=100)

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=False)

for g in summary_ids:
    sum = tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    sum = pp.pformat(sum)
    sum = re.sub("'", "", sum)
    sum = re.sub("\(", "", sum)
    sum = re.sub("\)", "", sum)
    
    print(sum)

Cheking model

In [ ]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="/content/drive/My Drive/RuBART/RuBART-Lenta",
    tokenizer="/content/drive/My Drive/RuBART/RuBART-Lenta",
)

In [ ]:
summarizer("""ЦИК обработал 100% протоколов. Против поправок выступили 21,27% россиян. 
Поправки к Конституции РФ на общероссийском голосовании поддержали 77,92% проголосовавших, следует из данных Центральной избирательной комиссии (ЦИК) РФ по итогам обработки 100% протоколов.
Согласно данным, опубликованным на экране в информационном центре ЦИК, против поправок к конституции выступили 21,27% проголосовавших. 
Общая явка, по последним данным, составила 65%.
Общероссийское голосование по поправкам к Конституции РФ завершилось на всей территории России в 21:00 мск в среду. 
""")